# Doing all the import stuff from Lab 4

In [1]:
#getting google drive mounted
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#A lot of the set up stuff here I copied from Lab4

# Installing openssh-server
!apt-get install openssh-server -qq > /dev/null

# Starting our server
!service ssh start

 * Starting OpenBSD Secure Shell server sshd
   ...done.


In [3]:
# Creating a new rsa key pair with empty password
!ssh-keygen -t rsa -P "" -f ~/.ssh/id_rsa

Generating public/private rsa key pair.
Created directory '/root/.ssh'.
Your identification has been saved in /root/.ssh/id_rsa
Your public key has been saved in /root/.ssh/id_rsa.pub
The key fingerprint is:
SHA256:9m3Gl5SJy1bo1082oTgQUsOSGHbDTeOo43lbCS7FGk4 root@1bff09535cc8
The key's randomart image is:
+---[RSA 3072]----+
|     o+o==       |
|    ...+=oo      |
|       o.o       |
|      o . .  o o |
|     E +S.  o =. |
|    + B...o=.+.o.|
|     * o o.oX.+.+|
|      o o  +.o oo|
|       .        .|
+----[SHA256]-----+


In [4]:
# Copying the public key we just generated to authorised keys
!cat $HOME/.ssh/id_rsa.pub>>$HOME/.ssh/authorized_keys

# Changing the permissions on the key
# Hint: Check "man chmod" for information on this command and remember, changing permissions
# so be careful with this command. 
!chmod 0600 ~/.ssh/authorized_keys

# Conneting with our local machine (essentially, we are just connecting to our own machine "as if" it is a remote server.)
# pptime will just tell us how long our system has been running. 
!ssh -o StrictHostKeyChecking=no localhost uptime

 17:07:38 up 1 min,  0 users,  load average: 2.17, 0.80, 0.29


In [5]:
#here is section 2 of Lab 4: Installing and configuring Hadoop

# Installing Hadoop and configuring JAVA_HOME: 
# Downloading Hadoop
# Upzipping 
# Copying hadoop into our /usr/local folder
# Removing the unused original copy
# Remove the compressed (zip) file, we're not using it anymore. 
# Adding a variable called "JAVA_HOME" to hadoop's environment script which tells it where Java is on our system. 

!if [ ! -d /usr/local/hadoop-3.3.3/ ]; then \
wget https://dlcdn.apache.org/hadoop/common/hadoop-3.3.3/hadoop-3.3.3.tar.gz; \
tar -xzf hadoop-3.3.3.tar.gz; \
cp -r hadoop-3.3.3/ /usr/local/; \
rm -rf hadoop-3.3.3/; \
rm hadoop-3.3.3.tar.gz; \
echo "export JAVA_HOME=$(dirname $(dirname $(realpath $(which java))))" >> /usr/local/hadoop-3.3.3/etc/hadoop/hadoop-env.sh; \
fi

--2023-03-09 17:07:38--  https://dlcdn.apache.org/hadoop/common/hadoop-3.3.3/hadoop-3.3.3.tar.gz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 645040598 (615M) [application/x-gzip]
Saving to: ‘hadoop-3.3.3.tar.gz’

hadoop-3.3.3.tar.gz 100%[===================>] 615.16M   155MB/s    in 3.9s    

2023-03-09 17:07:42 (158 MB/s) - ‘hadoop-3.3.3.tar.gz’ saved [645040598/645040598]



In [6]:
import os

# Setting up some of our environmental variables: 
# Here we add Hadoop's location to our path (in Python) and tell our system where hadoop is located. 
os.environ['PATH'] = "/usr/local/hadoop-3.3.3/bin/:" + os.environ['PATH']
os.environ["HADOOP_HOME"] = "/usr/local/hadoop-3.3.3"

In [7]:
# Here, we've uploaded lab4_config.sh to our Google drive , not that this path will depend on **YOUR** google drive. 
# We are then copying this to our "content" folder. 
# This does a lot of the annoying configuration for us. 
!cp /content/drive/MyDrive/project/lab4_config.sh /content/
!cat lab4_config.sh

#!/bin/bash

# Config for core-site.xml
sed -i '/<configuration>/a\
  <property>\n\
    <name>fs.defaultFS</name>\n\
    <value>hdfs://localhost:9000</value>\n\
  </property>' \
$HADOOP_HOME/etc/hadoop/core-site.xml

# Config for hdfs-site.xml 
# Since we are running Hadoop in only one machine, a replication factor >1 does not make sense 
sed -i '/<configuration>/a\
  <property>\n\
    <name>dfs.replication</name>\n\
    <value>1</value>\n\
  </property>' \
$HADOOP_HOME/etc/hadoop/hdfs-site.xml

# Config for mapred-site.xml 
sed -i '/<configuration>/a\
  <property>\n\
    <name>mapreduce.framework.name</name>\n\
    <value>yarn</value>\n\
  </property>\n\
  <property>\n\
    <name>mapreduce.application.classpath</name>\n\
    <value>$HADOOP_MAPRED_HOME/share/hadoop/mapreduce/*:$HADOOP_MAPRED_HOME/share/hadoop/mapreduce/lib/*</value>\n\
  </property>' \
$HADOOP_HOME/etc/hadoop/mapred-site.xml

# Config for yarn-site.xml
sed -i '/<configuration>/a\
  <property>\n\
    <description>The ho

In [8]:
# Running our config script
# Note: remember to check you have the correct filepath
!bash lab4_config.sh

In [9]:
!$HADOOP_HOME/bin/hdfs namenode -format

# Creating our HDFS environment variables: 
os.environ["HDFS_NAMENODE_USER"] = "root"
os.environ["HDFS_DATANODE_USER"] = "root"
os.environ["HDFS_SECONDARYNAMENODE_USER"] = "root"
os.environ["YARN_RESOURCEMANAGER_USER"] = "root"
os.environ["YARN_NODEMANAGER_USER"] = "root"

2023-03-09 17:08:04,975 INFO namenode.NameNode: STARTUP_MSG: 
/************************************************************
STARTUP_MSG: Starting NameNode
STARTUP_MSG:   host = 1bff09535cc8/172.28.0.12
STARTUP_MSG:   args = [-format]
STARTUP_MSG:   version = 3.3.3
STARTUP_MSG:   classpath = /usr/local/hadoop-3.3.3/etc/hadoop:/usr/local/hadoop-3.3.3/share/hadoop/common/lib/curator-client-4.2.0.jar:/usr/local/hadoop-3.3.3/share/hadoop/common/lib/httpcore-4.4.13.jar:/usr/local/hadoop-3.3.3/share/hadoop/common/lib/reload4j-1.2.18.3.jar:/usr/local/hadoop-3.3.3/share/hadoop/common/lib/jetty-server-9.4.43.v20210629.jar:/usr/local/hadoop-3.3.3/share/hadoop/common/lib/gson-2.8.9.jar:/usr/local/hadoop-3.3.3/share/hadoop/common/lib/jersey-json-1.19.jar:/usr/local/hadoop-3.3.3/share/hadoop/common/lib/jackson-core-asl-1.9.13.jar:/usr/local/hadoop-3.3.3/share/hadoop/common/lib/token-provider-1.0.1.jar:/usr/local/hadoop-3.3.3/share/hadoop/common/lib/curator-recipes-4.2.0.jar:/usr/local/hadoop-3.3.3/s

In [10]:
# Launching hdfs daemons
!$HADOOP_HOME/sbin/start-dfs.sh
# Listing the running daemons
!jps

Starting namenodes on [localhost]
Starting datanodes
Starting secondary namenodes [1bff09535cc8]
1bff09535cc8: Warning: Permanently added '1bff09535cc8,172.28.0.12' (ECDSA) to the list of known hosts.
1588 SecondaryNameNode
1398 DataNode
1833 Jps
1292 NameNode


In [11]:
!nohup $HADOOP_HOME/sbin/start-yarn.sh
!jps

nohup: ignoring input and appending output to 'nohup.out'
2066 NodeManager
1588 SecondaryNameNode
1956 ResourceManager
1398 DataNode
1292 NameNode
2190 Jps


# Copying Files and getting things ready (my own code)

In [12]:
#bringing in stopwords file from drive into current directory
!cp /content/drive/MyDrive/project/stopwords.txt /content/
!mkdir articles
#copying all article files from drive to current directory
!cp /content/drive/MyDrive/project/wikipedia-ml/article* /content/articles/

#removing white space to put all files in hadoop
#hadoop didnt like it when there were spaces in the file name
#inspiration for this: https://stackoverflow.com/questions/2709458/how-to-replace-spaces-in-file-names-using-a-bash-script
!for f in articles/*\ *; do mv "$f" "${f// /_}"; done



In [13]:
#making hadoop directory and adding all article files into it
!$HADOOP_HOME/bin/hdfs dfs -mkdir /hadoop_articles
!$HADOOP_HOME/bin/hdfs dfs -put /content/articles/*.txt /hadoop_articles
!$HADOOP_HOME/bin/hdfs dfs -ls /hadoop_articles/

Found 21 items
-rw-r--r--   1 root supergroup      12305 2023-03-09 17:09 /hadoop_articles/article_Machine_learning_-_Wikipedia_2013_1_2.txt
-rw-r--r--   1 root supergroup      12048 2023-03-09 17:09 /hadoop_articles/article_Machine_learning_-_Wikipedia_2013_6_27.txt
-rw-r--r--   1 root supergroup      12391 2023-03-09 17:09 /hadoop_articles/article_Machine_learning_-_Wikipedia_2014_1_4.txt
-rw-r--r--   1 root supergroup      12396 2023-03-09 17:09 /hadoop_articles/article_Machine_learning_-_Wikipedia_2014_6_4.txt
-rw-r--r--   1 root supergroup      16236 2023-03-09 17:09 /hadoop_articles/article_Machine_learning_-_Wikipedia_2015_1_4.txt
-rw-r--r--   1 root supergroup      17678 2023-03-09 17:09 /hadoop_articles/article_Machine_learning_-_Wikipedia_2015_6_30.txt
-rw-r--r--   1 root supergroup      17526 2023-03-09 17:09 /hadoop_articles/article_Machine_learning_-_Wikipedia_2016_1_1.txt
-rw-r--r--   1 root supergroup      19147 2023-03-09 17:09 /hadoop_articles/article_Machine_learning_

# Co Occurence

Python files for co occurence

In [14]:
%%writefile mapper_co_occur.py

#!/usr/bin/env python3

import sys


with open("stopwords.txt") as s:
    stopwords = set([line.strip() for line in s])


delimiter = '\t'

for line in sys.stdin:
    words = line.strip().split()

    # Iterate over each word in the line
    for i in range(len(words)):
        # If the word is a stopword, skip it
        if words[i] in stopwords:
            continue

        # Iterate over the words in the window
        for j in range(i + 1, len(words)):
            # If the word is a stopword, skip it
            if words[j] in stopwords:
                continue

            # Output the co-occurrence pair
            key = ("("+words[i] + ", " + words[j] + ")")
            print('%s\t%s' % (key, 1))

Writing mapper_co_occur.py


In [15]:
%%writefile reducer_co_occur.py

#!/usr/bin/env python3

import sys
delimiter = '\t'

current_key = None
current_count = 0
key = None


for line in sys.stdin:
    line = line.replace(".", "").replace(",", "").replace("?","").replace("!","")

    key, count = line.strip().split(delimiter)
    

    try:
        count = int(count)  # convert count (currently a string) to int
    except ValueError:
        continue  # silently ignore line if count is not a number

    if current_key == key:
        current_count += count
    else:
        if current_key:
            print('%s\t%s' % (current_key, current_count))
        current_count = count
        current_key = key

if current_key == key:
    print('%s\t%s' % (current_key, current_count))

Writing reducer_co_occur.py


Running co_occur

In [16]:
#Running MapReduce programs
!$HADOOP_HOME/bin/hadoop jar $HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-3.3.3.jar \
  -files /content/stopwords.txt \
  -input /hadoop_articles/*.txt \
  -output /coocur-outputs \
  -mapper "python /content/mapper_co_occur.py" \
  -reducer "python /content/reducer_co_occur.py"

packageJobJar: [/tmp/hadoop-unjar3304929965359211289/] [] /tmp/streamjob11639250981694717338.jar tmpDir=null
2023-03-09 17:09:15,381 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at localhost/127.0.0.1:8032
2023-03-09 17:09:15,647 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at localhost/127.0.0.1:8032
2023-03-09 17:09:16,080 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1678381730565_0001
2023-03-09 17:09:17,008 INFO mapred.FileInputFormat: Total input files to process : 21
2023-03-09 17:09:17,224 INFO mapreduce.JobSubmitter: number of splits:21
2023-03-09 17:09:17,725 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1678381730565_0001
2023-03-09 17:09:17,725 INFO mapreduce.JobSubmitter: Executing with tokens: []
2023-03-09 17:09:18,040 INFO conf.Configuration: resource-types.xml not found
2023-03-09 17:09:18,040 INFO resource.ResourceUtils: U

Output from co-occurence mapreduce

In [17]:
!$HADOOP_HOME/bin/hdfs dfs -cat /coocur-outputs/part-00000

Streaming output truncated to the last 5000 lines.
(outputs These)	3
(outputs Transduction)	6
(outputs When)	3
(outputs acquire)	6
(outputs active)	3
(outputs algorithm)	18
(outputs algorithms)	27
(outputs also)	3
(outputs applications)	6
(outputs are)	6
(outputs area)	6
(outputs autonomous)	3
(outputs based)	6
(outputs bias)	3
(outputs budget)	3
(outputs can)	6
(outputs case)	6
(outputs choice)	3
(outputs classification)	21
(outputs closely)	6
(outputs computer)	3
(outputs continuous)	6
(outputs cover)	3
(outputs cumulatively)	3
(outputs curriculum)	3
(outputs density)	6
(outputs developmental)	3
(outputs documents)	6
(outputs dynamic)	3
(outputs either)	3
(outputs email)	30
(outputs emails)	18
(outputs entire)	6
(outputs environment)	3
(outputs estimation)	3
(outputs example)	12
(outputs examples)	6
(outputs except)	6
(outputs experience)	3
(outputs experiences)	3
(outputs exploration)	3
(outputs face)	6
(outputs false)	2
(outputs feedback)	3
(outputs file)	15
(outputs filters)	15
(o

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



# Bigram

Python files for bigram

In [18]:
%%writefile mapper_bigram.py

#!/usr/bin/env python3

import sys

delimiter = '\t'

for line in sys.stdin:
    line = line.replace(".", "").replace(",", "").replace("?","").replace("!","")
    words = line.strip().split()
    for i in range(len(words) -1):
        key = ("("+words[i] + ", " + words[i+1] + ")")
        print('%s\t%s' % (key, 1))


Writing mapper_bigram.py


In [19]:
%%writefile reducer_bigram.py
#!/usr/bin/env python3

import sys

current_key = None
current_count = 0
key = None
delimiter = "\t"


for line in sys.stdin:
    key, count = line.strip().split(delimiter)
    

    try:
        count = int(count)  # convert count (currently a string) to int
    except ValueError:
        continue  # silently ignore line if count is not a number

    if current_key == key:
        current_count += count
    else:
        if current_key:
            print('%s\t%s' % (current_key, current_count))
        current_count = count
        current_key = key

if current_key == key:
    print('%s\t%s' % (current_key, current_count))

Writing reducer_bigram.py


In [20]:
%%writefile combiner_bigram.py
#!/usr/bin/env python3

import sys

current_key = None
current_count = 0
key = None
delimiter = "\t"


for line in sys.stdin:
    key, count = line.strip().split(delimiter)
    
    try:
        count = int(count)  # convert count (currently a string) to int
    except ValueError:
        continue  # silently ignore line if count is not a number

    if current_key == key:
        current_count += count
    else:
        if current_key:
            print('%s\t%s' % (current_key, current_count))
        current_count = count
        current_key = key

if current_key == key:
    print('%s\t%s' % (current_key, current_count))

Writing combiner_bigram.py


Running Map Reduce Bigram

In [21]:
#Running MapReduce programs
!$HADOOP_HOME/bin/hadoop jar $HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-3.3.3.jar \
  -input /hadoop_articles/*.txt \
  -output /bigram-outputs \
  -mapper "python /content/mapper_bigram.py" \
  -combiner "python /content/combiner_bigram.py" \
  -reducer "python /content/reducer_bigram.py"

packageJobJar: [/tmp/hadoop-unjar18248572447454038624/] [] /tmp/streamjob16651194598314540286.jar tmpDir=null
2023-03-09 17:12:26,635 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at localhost/127.0.0.1:8032
2023-03-09 17:12:27,234 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at localhost/127.0.0.1:8032
2023-03-09 17:12:27,792 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1678381730565_0002
2023-03-09 17:12:28,189 INFO mapred.FileInputFormat: Total input files to process : 21
2023-03-09 17:12:28,798 INFO mapreduce.JobSubmitter: number of splits:21
2023-03-09 17:12:29,588 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1678381730565_0002
2023-03-09 17:12:29,588 INFO mapreduce.JobSubmitter: Executing with tokens: []
2023-03-09 17:12:29,870 INFO conf.Configuration: resource-types.xml not found
2023-03-09 17:12:29,871 INFO resource.ResourceUtils: 

Bigram output

In [23]:
!$HADOOP_HOME/bin/hdfs dfs -cat /bigram-outputs/part-00000

Streaming output truncated to the last 5000 lines.
(either, supervised)	22
(either, within)	9
(either, women)	4
(elaborated, for)	8
(electrically, adjustable)	6
(electrocardiograms, and)	2
(eliminate, the)	4
(eliminated, since)	4
(elimination, or)	4
(email, For)	3
(email, and)	9
(email, as)	1
(email, filtering)	12
(email, message)	1
(email, messages)	8
(emails, the)	12
(embedded, devices)	2
(embedded, systems)	2
(emotion, The)	7
(emotion, toward)	7
(emotions, about)	7
(emotions, feelings)	7
(emphasis, on)	17
(emphasize, the)	3
(employ, the)	21
(employed, especially)	17
(employed, in)	10
(employed, today)	9
(employee, and)	2
(employs, data)	21
(employs, various)	6
(emulate, a)	3
(emulate, neurons)	3
(emulate, the)	3
(enables, it)	19
(encoding, of)	21
(encompasses, a)	7
(encompasses, other)	9
(encountered, in)	7
(endeavor, machine)	7
(endeavour, machine)	10
(engine, accordingly)	16
(engineering, and)	9
(engineering, example)	1
(engineering, teams)	2
(engineers, and)	5
(engineers, need)	2